In [ ]:
import os
new_directory = '/home/fruitspec-lab/FruitSpec/Code/roi/fsCounter'
os.chdir(new_directory)
current_directory = os.getcwd()
print("Current working directory:", current_directory)

In [ ]:
from MHS.F_model_training import *
from vision.misc.help_func import go_up_n_levels
from vision.pipelines.fe_pipeline import *
from vision.feature_extractor.vegetation_indexes import num_deno_nan_divide, num_deno_nan_divide_np, ndvi_cuda

# os.chdir(r'/home/fruitspec-lab/FruitSpec/Code/roi/fsCounter/MHS')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from vision.tools.image_stitching import plot_2_imgs
from vision.tools.camera import is_saturated

In [ ]:
# {0: (0, 100), 1: (25, 255), 2: (0, 150)},
def thresh_channels(frame, xyz_frame, channels_thresh={0: (0, 175), 1: (10, 255), 2: (0, 200)},
                    gree_minus_red_thres=(0, 0), gree_minus_blue_thres=(0, 0), vari_filter = 5, vndvi_filter = 0.35,
                    depth_filter = (1, 5),show_imgs=False):
    """
    Thresholds the input frame based on the given channel thresholds and green minus red thresholds.

    Args:
        frame (ndarray): The input frame to threshold.
        channels_thresh (dict): A dictionary containing the channel indices as keys and a tuple of thresholds as values.
        gree_minus_red_thres (tuple): A tuple containing the green minus red thresholds.

    Returns:
        ndarray: The thresholded frame.

    """
    frame = np.copy(frame)
    if np.sum(depth_filter) > 0:
        if show_imgs:
            frame_pre = frame.copy()
        frame[xyz_frame[:,:,2] < depth_filter[0]] = 0
        frame[xyz_frame[:,:,2] > depth_filter[1]] = 0
        if show_imgs:
            plot_2_imgs(frame_pre, frame, title = "pre depth filter,    post filter")        
    if vari_filter + vndvi_filter > 0:
        Red, Green, Blue = frame[:,:,0], frame[:,:,1], frame[:,:,2]
#     if show_imgs:
#         vndvi[~np.isfinite(vndvi)] = np.nanmin(vndvi)
#         plot_2_imgs(vari, vndvi, title = "Vari, vNDVI")
#         plot_2_imgs(vari > 0, vndvi > 0, title = "Vari, vNDVI")
#         plot_2_imgs(vari > 1, vndvi > 0.3, title = "Vari, vNDVI")
#         plot_2_imgs(vari > 2, vndvi > 0.31, title = "Vari, vNDVI")
#         plot_2_imgs(vari > 3, vndvi > 0.32, title = "Vari, vNDVI")
#         plot_2_imgs(vndvi > 0.35, vndvi > 0.33, title = "vNDVI>0.35, vNDVI>0.33")
#         plot_2_imgs(vari > 4.5, vndvi > 0.34, title = "Vari, vNDVI")
#         plot_2_imgs(vari >5, vndvi > 0.35, title = "Vari, vNDVI")
#         plot_2_imgs(frame[:,:,0] - frame[:,:,2], frame[:,:,2] - frame[:,:,0])
#         plot_2_imgs(frame[:,:,1] - frame[:,:,0], frame[:,:,1] - frame[:,:,2])
#         plot_2_imgs(frame[:,:,1] - frame[:,:,0]>10, frame[:,:,1] - frame[:,:,2]>10)
#         plot_2_imgs(frame[:,:,1] - frame[:,:,0]>50, frame[:,:,1] - frame[:,:,2]>50)
#         plot_2_imgs(frame[:,:,1] - frame[:,:,0]>150, frame[:,:,1] - frame[:,:,2]>150)
#         plot_2_imgs(frame[:,:,1] - frame[:,:,0]>250, frame[:,:,1] - frame[:,:,2]>250)
    for channel, thresholds in channels_thresh.items():
        if not np.sum(thresholds):
            continue
        chan = frame[:, :, channel]
        valids = cv2.inRange(chan, thresholds[0], thresholds[1])
        frame[np.where(valids==0)] = 0

    if np.sum(gree_minus_red_thres) != 0:
        diff = frame[:, :, 1] - frame[:, :, 0]
        valids = cv2.inRange(diff, gree_minus_red_thres[0], gree_minus_red_thres[1])
        if show_imgs:
            frame_pre = frame.copy()
        frame[np.where(valids==0)] = 0
        if show_imgs:
            plot_2_imgs(frame_pre, frame, title = "prefilter g minus r,    post filter")
            
    if np.sum(gree_minus_blue_thres) !=0:
        diff = frame[:, :, 1] - frame[:, :, 2]
        valids = cv2.inRange(diff, gree_minus_blue_thres[0], gree_minus_blue_thres[1])
        if show_imgs:
            frame_pre = frame.copy()
        frame[np.where(valids==0)] = 0
        if show_imgs:
            plot_2_imgs(frame_pre, frame, title = "prefilter g minus b,    post filter")
    
    if vari_filter > 0:
        vari = num_deno_nan_divide_np((Green - Red), (Green + Red - Blue))
        vari[~np.isfinite(vari)] = 0
        vari = np.clip(vari, -5,5)
        if show_imgs:
            frame_pre = frame.copy()
        frame_copy = frame.copy()
        frame_copy[vari >= vari_filter]= 0
        frame[-int(frame.shape[0]/4):,:] = frame_copy[-int(frame.shape[0]/4):,:]
        if show_imgs:
            plot_2_imgs(frame_pre, frame, title = "prefilter vari,    post filter")
            
    if vndvi_filter > 0:
        vndvi = 0.5268 * (Red**(-0.1294) * Green**(0.3389)* Blue**(-0.3118))
        if show_imgs:
            frame_pre = frame.copy()
        frame[vndvi <= vndvi_filter] = 0
        if show_imgs:
            plot_2_imgs(frame_pre, frame, title = "prefilter vndvi,    post filter")
            
    if show_imgs:
        if not vari_filter:
            vari = num_deno_nan_divide_np((Green - Red), (Green + Red - Blue))
            vari[~np.isfinite(vari)] = 0
            vari = np.clip(vari, -5,5)   
        if not vndvi_filter:
            vndvi = 0.5268 * (Red**(-0.1294) * Green**(0.3389)* Blue**(-0.3118))
        plot_2_imgs(vari < vari_filter, vndvi > vndvi_filter, title = "vari filtered,    ndvi filtred")
    return cv2.threshold(frame[:, :, 1], 0, 255, cv2.THRESH_BINARY)[1] > 0


def get_percent_seen(zed_frame, xyz_frame, coors, return_threshed=False, show_imgs=False):
    """
    Calculates the percent of the frame that is covered by the input coordinates.

    Args:
        zed_frame (ndarray): The input frame.
        coors (tuple): A tuple of coordinates (x1, y1, x2, y2) to exclude from the calculation.

    Returns:
        float: The percent of the frame that is covered by the input coordinates.
    """
    x1, y1, x2, y2 = coors
    zed_threshed = thresh_channels(zed_frame, xyz_frame, show_imgs=show_imgs)
    zed_threshed_cut = zed_threshed[:, x1:x2]
    percent_seen = round(np.sum(zed_threshed_cut[y1:y2, :])/np.sum(zed_threshed_cut),2)
    if return_threshed:
        return percent_seen, zed_threshed_cut
    return percent_seen

def debug_percent_seen_batch(batch_fsi, batch_rgb_zed, batch_zed, frame_ids, b_align):
    for jai, zed, zed_xyz, f_id, cors in zip(batch_fsi, batch_rgb_zed, batch_zed, frame_ids, b_align):
        cors = [int(cor) for cor in cors[:4]]
        x1, y1, x2, y2 = cors
        zed_copy = zed.copy()
        color = (0, 0, 255)
        thickness = 2
        percent_seen, zed_threshed = get_percent_seen(zed_copy, zed_xyz, cors[:4], True, True)
        cv2.rectangle(zed_copy, (x1, y1), (x2, y2), color, thickness)
        plot_2_imgs(jai, zed_copy, title=f_id, save_to="", save_only=False, cv2_save=False, quick_save=False)
        plot_2_imgs(zed_threshed, zed_copy, title=f"frame: {f_id}, percent: {percent_seen}",
                    save_to="", save_only=False, cv2_save=False, quick_save=False)


In [ ]:
def run_on_row(row_scan_path, min_slice_len, direction, tree_id, frames, shift):
    adts_loader = None
    validate_jai_zed_json(row_scan_path)
    validate_slice_data(row_scan_path, min_slice_len=min_slice_len, direction=direction)
    metadata, _ = get_metadata(row_scan_path)
    fe, adts_loader, batch_size = init_fe_obj(row_scan_path, tree_id, adts_loader, False, direction)
    n_batchs = len(frames) // batch_size
    for i in tqdm(range(n_batchs)):
        frame_ids = frames[i * batch_size: (i + 1) * batch_size]
        batch_res = adts_loader.load_batch(frame_ids, shift)
        batch_fsi, batch_zed, batch_jai_rgb, batch_rgb_zed, batch_tracker, batch_slicer, frame_ids, b_align, b_jai_translation = batch_res
        debug_percent_seen_batch(batch_fsi, batch_rgb_zed, batch_zed, frame_ids, b_align)
        

In [ ]:
min_slice_len=5
direction="left"
frames = [str(num) for num in range(100,110)]
tree_id = 0
shift = 0
os.chdir(new_directory)

In [ ]:
os.chdir(new_directory)
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/BEERAMU0/220823/row_107/1"
frames = [str(num) for num in range(100,109)]
run_on_row(row_scan_path, min_slice_len, direction, tree_id, frames, shift)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/ORSCHIST/230813/row_6/1"
frames = [str(num) for num in range(250,259)]
run_on_row(row_scan_path, min_slice_len, direction, tree_id, frames, shift)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/MICHAL22/240723/row_102/2"
frames = [str(num) for num in range(200,209)]
run_on_row(row_scan_path, min_slice_len, "right", tree_id, frames, shift)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/RAUSTENB/060723/row_2/1"
frames = [str(num) for num in range(400,409)]
run_on_row(row_scan_path, min_slice_len, "right", tree_id, frames, shift)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/MEIRAVHA/230723/row_101/1"
frames = [str(num) for num in range(100,109)]
run_on_row(row_scan_path, min_slice_len, "right", tree_id, frames, shift)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/PROPAL/M7XXXXXX/250323/row_23/1"
frames = [str(num) for num in range(1000,1009)]
run_on_row(row_scan_path, min_slice_len, "right", tree_id, frames, shift)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/PROPAL/FUKUMOTO/260323/row_22/1"
frames = [str(num) for num in range(2000, 2009)]
run_on_row(row_scan_path, min_slice_len, "right", tree_id, frames, shift)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/PROPAL/2018XXXX/230323/row_62/1"
frames = [str(num) for num in range(1500, 1509)]
run_on_row(row_scan_path, min_slice_len, "right", tree_id, frames, shift)